In [ ]:
import numpy as np
A = np.arange(4).reshape((2, 2))
d = np.array((1, 2))
D = np.diag(d)
B = D.dot(A)
d2 = np.expand_dims(d, -1)
C = d2 * A
E = np.dot(A, d)

# now try multiplying the same rank 2 matrix by a stack of vectors
I = np.eye(2)
I[1,1] += 1
B = np.arange(20).reshape((-1, 2))
G = np.einsum('jk,ik->ji', B, I)

# now try multiplying a stack of matrices by a stack of matrices
F = np.tile(I, (5,1,1))
for i in range(5):
    F[i,1,0] += i
H = np.einsum('ijk,ilk->ijl', F, F)

# now try multiplying a stack of matrices by a stack of diagonal (vectors, but interpreted as matrices)
D2 = np.tile(np.array((2, 1)), (5, 1))
U = np.einsum('ijk,ik->ijk', F, D2)

# now try multiplying by the diagonal
ones = np.ones(4).reshape((2, 2))
L = np.einsum('ij,j->ij', ones, d)

# PRINTS
print(H)

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import tensorflow as tf
import numpy as np
A = tf.constant(np.arange(8).reshape((2, 2, 2)))
d = tf.constant(np.arange(4).reshape((2, 2)) + 1)
D = tf.linalg.diag(d)
B = tf.matmul(D, A)
d2 = tf.expand_dims(d, -1)
C = d2 * A
E = tf.einsum('ijk,ik->ij', A, d)

# test making a stacked identity
I_ = tf.expand_dims(tf.eye(3), 0)
I = tf.tile(I_, (5, 1, 1))

# testing matrix_band_part
G = tf.constant(np.random.normal(size=(5, 3, 3)))
G_ld = tf.matrix_band_part(G, -1, 0) - tf.matrix_band_part(G, 0, 0)
G_ud = tf.einsum('ijk->ikj', G_ld)

with tf.Session() as sess:
    items = sess.run([G_ld, G_ud])
    
for i in items:
    print(i)
    print(i.shape)
    print('-' * 15)

In [ ]:
# TEST IF eigh() IS DIFFERENTIABLE
A = tf.Variable(np.random.normal(size=(5, 5)))
A = tf.matmul(A, A, transpose_b=True)
e, v = tf.linalg.eigh(A)
loss = tf.linalg.norm(tf.einsum('ij,kj->ik', tf.einsum('ij,j->ij', v, e), v) - tf.cast(tf.eye(5), tf.float64))
e, v = e[::-1], v[:, ::-1]
error = tf.linalg.norm(tf.einsum('ij,kj->ik', tf.einsum('ij,j->ij', v, e), v) - A)

opt = tf.train.AdamOptimizer()
step = opt.minimize(loss)

# initialize
with tf.Session() as sess:
    sess.run(tf.initializers.global_variables())
    for i in range(10000):
        loss_, error_, _ = sess.run([loss, error, step])
        if i % 1000 == 0:
            print(i, loss_, error_)
        
    A_ = sess.run(A)
    
print(A_)

In [ ]:
# TEST MATRIX STACK x MATRIX STACK
n_stacks = 10
n = 10

# generate matrices
matrices = []
products = []
for i in range(n_stacks):
    matrix = np.random.normal(size=(n, n))
    product = np.dot(matrix, matrix.T)
    matrices.append(matrix)
    products.append(product)
    
matrix_stack = np.concatenate([np.expand_dims(m, axis=0) for m in matrices], axis=0)
product_stack = np.einsum('ijk,ilk->ijl', matrix_stack, matrix_stack)

diffs = []
for i in range(n_stacks):
    diff = np.sum(products[i] - product_stack[i,:,:])
    diffs.append(diff)
    
print(sum(diffs))

In [ ]:
# TEST MATRIX STACK x VECTOR STACK (REPRESENTING DIAGONALS)
n_stacks = 10
n = 10

# generate matrices
matrices = []
diagonals = []
products = []
for i in range(n_stacks):
    matrix = np.random.normal(size=(n, n))
    diagonal = np.random.normal(size=(n,))
    product = np.dot(matrix, np.diag(diagonal))
    matrices.append(matrix)
    diagonals.append(diagonal)
    products.append(product)
    
matrix_stack = np.concatenate([np.expand_dims(m, axis=0) for m in matrices], axis=0)
diagonals_stack = np.concatenate([np.expand_dims(d, axis=0) for d in diagonals], axis=0)
product_stack = np.einsum('ijk,ik->ijk', matrix_stack, diagonals_stack)

diffs = []
for i in range(n_stacks):
    diff = np.sum(products[i] - product_stack[i,:,:])
    diffs.append(diff)
    
print(sum(diffs))

In [ ]:
# TEST MATRIX STACK x VECTOR STACK (REPRESENTING ACTUAL VECTORS)
n_stacks = 10
n = 10

# generate matrices
matrices = []
vectors = []
products = []
for i in range(n_stacks):
    matrix = np.random.normal(size=(n, n))
    vector = np.random.normal(size=(n,))
    product = np.dot(matrix, vector)
    matrices.append(matrix)
    vectors.append(vector)
    products.append(product)
    
matrix_stack = np.concatenate([np.expand_dims(m, axis=0) for m in matrices], axis=0)
vectors_stack = np.concatenate([np.expand_dims(v, axis=0) for v in vectors], axis=0)
product_stack = np.einsum('ijk,ik->ij', matrix_stack, vectors_stack)

diffs = []
for i in range(n_stacks):
    diff = np.sum(products[i] - product_stack[i])
    diffs.append(diff)
    
print(sum(diffs))